In [5]:
# -*- coding: utf-8 -*-
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout
from keras.optimizers import SGD
import math


# 이미지 데이터 경로 설정
train_path = r'/content/drive/MyDrive/졸프데이터'
valid_path = r'/content/drive/MyDrive/validationset'

test_path = r'/content/drive/MyDrive/testset'

# 이미지 크기 설정
img_width, img_height = 224, 224

# 데이터 제너레이터 생성
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path, target_size=(img_width, img_height), batch_size=32, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_path, target_size=(img_width, img_height), batch_size=32, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path, target_size=(img_width, img_height), batch_size=32, class_mode='categorical')

# VGG16 모델 로딩 및 학습 레이어 제거
input_tensor = Input(shape=(img_width, img_height, 3))
vgg16_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
last_layer = vgg16_model.output
last_layer = Flatten()(last_layer)
last_layer = Dense(256, activation='relu')(last_layer)
last_layer = Dropout(0.5)(last_layer)
output_layer = Dense(train_generator.num_classes, activation='softmax')(last_layer)

model = Model(inputs=vgg16_model.input, outputs=output_layer)

# 미세 조정을 위한 학습 설정
for layer in model.layers[:15]:
    layer.trainable = False

sgd = SGD(lr=0.001, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


# 모델 학습 및 평가
steps_per_epoch = math.floor(train_generator.samples / train_generator.batch_size)
validation_steps = math.floor(valid_generator.samples / valid_generator.batch_size)
test_steps = math.ceil(test_generator.samples / test_generator.batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, validation_data=valid_generator, validation_steps=validation_steps)

scores = model.evaluate_generator(test_generator, steps=test_steps)
print("Test accuracy: %.2f%%" % (scores[1]*100))



#이미지 넣어서 테스트하기
from tensorflow.keras.preprocessing import image
import numpy as np

# 이미지 경로 설정
img_path = r'/content/drive/MyDrive/testset'

# 이미지 불러오기
img = image.load_img(img_path, target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

# 예측하기
prediction = model.predict(x)
class_idx = np.argmax(prediction)
class_label = train_generator.class_indices
predicted_label = list(class_label.keys())[list(class_label.values()).index(class_idx)]
print('Predicted class:', predicted_label)

Found 5000 images belonging to 5 classes.
Found 401 images belonging to 5 classes.
Found 0 images belonging to 0 classes.


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
<ipython-input-5-79263ab8ce2d>:52: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, validation_data=valid_generator, validation_steps=validation_steps)


Epoch 1/5
156/156 [==============================] - 2682s 17s/step - loss: 0.9483 - accuracy: 0.6258 - val_loss: 0.4752 - val_accuracy: 0.8255
Epoch 2/5
156/156 [==============================] - 2662s 17s/step - loss: 0.3857 - accuracy: 0.8621 - val_loss: 0.3061 - val_accuracy: 0.8828
Epoch 3/5
156/156 [==============================] - 2703s 17s/step - loss: 0.2546 - accuracy: 0.9149 - val_loss: 0.3313 - val_accuracy: 0.8880
Epoch 4/5
156/156 [==============================] - 2713s 17s/step - loss: 0.1929 - accuracy: 0.9273 - val_loss: 0.2442 - val_accuracy: 0.9167
Epoch 5/5
156/156 [==============================] - 2709s 17s/step - loss: 0.1376 - accuracy: 0.9527 - val_loss: 0.2878 - val_accuracy: 0.9089


<ipython-input-5-79263ab8ce2d>:54: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, steps=test_steps)


ValueError: ignored

In [8]:
# Save the trained model in the models directory
model.save('food_classifier_vgg16.h5')


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
